In [1]:
import queue
import sys

import sounddevice as sd
import soundfile as sf
import numpy  # Make sure NumPy is loaded before it is used in the callback
assert numpy  # avoid "imported but unused" message (W0611)

from situtils import FPSTimes

import threading

import time

## Class method version

In [2]:
%%writefile microphones.py 
#^IMPORTANT: essential to make multiprocessing work

import queue
import sys

import sounddevice as sd
import soundfile as sf
import numpy  # Make sure NumPy is loaded before it is used in the callback
assert numpy  # avoid "imported but unused" message (W0611)

from situtils import FPSTimes

import time

class MicrophoneController(FPSTimes):
    # https://python-sounddevice.readthedocs.io/en/0.3.15/examples.html#recording-with-arbitrary-duration
    
    @staticmethod
    def callback(indata, frames, time, status):
        """This is called (from a separate thread) for each audio block."""
        if status:
            print(status, file=sys.stderr)
        MicrophoneController.queue.put(indata.copy())
        
        
    @classmethod
    def run(cls, status, cfg):
        # MicrophoneController.initialize(cfg)
        print("Running.")
        import sounddevice as sd  # must be inside the function
        
        if cfg['channel_selectors']: # make it work without ASIO
            # https://python-sounddevice.readthedocs.io/en/0.3.15/api/platform-specific-settings.html
            asio_in = sd.AsioSettings(channel_selectors=cfg['channel_selectors'])
        else:
            asio_in = None

        MicrophoneController.queue = queue.Queue() # kind of a hack

        stream = sd.InputStream(samplerate=cfg['sample_rate'], device=cfg['device'], channels=cfg['number_channels'], callback=MicrophoneController.callback, extra_settings = asio_in)
 
        filename = cfg['file_path']
        file = sf.SoundFile(filename, mode='w', samplerate=cfg['sample_rate'], channels=cfg['number_channels'],subtype='PCM_32') # 'w': overwrite mode, 'x': raises error if file exists

        # experiment status: 1 - idle, 2 - running (recording, logging), 0 - stopped
        with file as f:
            while status.value > 0:
                try:
                    if status.value == 2:

                        # start stream if not active yet
                        if not stream.active:
                            print("Audio input stream started.")
                            stream.start()

                        f.write(MicrophoneController.queue.get())

                    else:
                        time.sleep(0.005)
                except KeyboardInterrupt:
                    stream.stop()
                    stream.close()
                    break
        

Overwriting microphones.py


## Instance version

In [22]:
%%writefile microphones_instance.py 
#^IMPORTANT: essential to make multiprocessing work

import queue
import sys

import sounddevice as sd
import soundfile as sf
import numpy  # Make sure NumPy is loaded before it is used in the callback
assert numpy  # avoid "imported but unused" message (W0611)

from situtils import FPSTimes

import time

import threading

class MicrophoneControllerInstance(FPSTimes):
    # https://python-sounddevice.readthedocs.io/en/0.3.15/examples.html#recording-with-arbitrary-duration
    def __init__(self, status, cfg):
        import sounddevice as sd  # must be inside the function? TODO
        self.cfg = cfg
        self.samplerate = cfg['sample_rate']
        self.device = cfg['device']
        self.channels = cfg['number_channels']
        self.status = status
        
        if cfg['channel_selectors']: # make it work without ASIO
            # https://python-sounddevice.readthedocs.io/en/0.3.15/api/platform-specific-settings.html
            self.channel_selectors = cfg['channel_selectors']
            asio_in = sd.AsioSettings(channel_selectors=self.channel_selectors)
        else:
            asio_in = None

        MicrophoneControllerInstance.queue = queue.Queue() # kind of a hack
        
        self.stream = sd.InputStream(samplerate=self.samplerate, device=self.device, channels=self.channels, callback=self.callback, extra_settings = asio_in)

        self.filename = cfg['file_path']
        self.file = sf.SoundFile(self.filename, mode='w', samplerate=self.samplerate, channels=self.channels) # 'w': overwrite mode, 'x': raises error if file exists
        
    
    def start(self):
        self._th = threading.Thread(target=self.run, args=())
        self._th.start()

    def stop(self):
        time.sleep(0.2)   # wait until device is released
        self._th.join()
        print('Microphones recording stopped')

    def start_stream(self):
        self.stream.start()

    def stop_stream(self):
        self.stream.stop()
        self.stream.close()

    # Used in all versions
    @staticmethod
    def callback(indata, frames, time, status):
        """This is called (from a separate thread) for each audio block."""
        if status:
            print(status, file=sys.stderr)
        MicrophoneControllerInstance.queue.put(indata.copy())
    
    # instance method
    def run(self):
        import sounddevice as sd  # must be inside the function? TODO
        # experiment status: 1 - idle, 2 - running (recording, logging), 0 - stopped
        with self.file as file:
            while self.status.value > 0:
                if self.status.value == 2:
                    
                    # start stream if not active yet
                    if not self.stream.active:
                        print("Audio input stream started.")
                        self.stream.start()
                    
                    file.write(MicrophoneControllerInstance.queue.get())
        
        self.stop_stream()

Writing microphones_instance.py


## Test microphones

Import config file

In [47]:
import json
import os
import multiprocess as mp

# cfg_filename = os.path.join('..','profiles', 'miguel_socialSIT_test.json')
cfg_filename = os.path.join('..','profiles', 'miguel_socialSIT_test_lord_sith.json')
with open(cfg_filename) as json_file:
    cfg = json.load(json_file)

cfg["microphones"]

{'record_audio': True,
 'sample_rate': 44000,
 'device': 1,
 'number_channels': 2,
 'channel_selectors': False,
 'file_path': 'audio_new.mat5'}

### Test version using instance methods and no parallelization -> Works!

In [9]:
# experiment status: 1 - idle, 2 - running (recording, logging), 0 - stopped
status = mp.Value('i', 1)

mc = MicrophoneControllerInstance(status, cfg["microphones"])
try:
    status.value = 2
    print("Recording started")
    mc.run()
except KeyboardInterrupt:
    status.value = 0
    mc.stop_stream()
    print("Recording stopped")


Recording started
Audio input stream started.
Recording stopped


### Test with class method instead of instance method -> Works!

In [22]:
from microphones import MicrophoneController
# experiment status: 1 - idle, 2 - running (recording, logging), 0 - stopped
status = mp.Value('i', 2)
MicrophoneController.run(status,cfg["microphones"])
print("Recording stopped")

Running.
Audio input stream started.


TypeError: write() argument must be str, not numpy.ndarray

### Test with multiprocessing using class method -> Works!

In [83]:
from microphones import MicrophoneController # IMPORTANT: class must be imported from separate .py file, not from the notebook

# experiment status: 1 - idle, 2 - running (recording, logging), 0 - stopped
status = mp.Value('i', 1)

mc = mp.Process(target=MicrophoneController.run, args=(status,cfg["microphones"]))
mc.start()

In [84]:
mc

<Process name='Process-12' pid=11076 parent=11092 started>

In [85]:
status.value = 2

print("Recording started")

Recording started


In [86]:
status.value = 0

mc.join()

### Test with multiprocessing using instance method -> Error!

In [23]:
from microphones_instance import MicrophoneControllerInstance

# experiment status: 1 - idle, 2 - running (recording, logging), 0 - stopped
status = mp.Value('i', 1)

microphoneController = MicrophoneControllerInstance(status, cfg["microphones"])

mc = mp.Process(target=microphoneController.run, args=())
mc.start()

TypeError: cannot pickle '_cffi_backend.__CDataOwnGC' object

In [ ]:
status.value = 2

print("Recording started")

In [ ]:
status.value = 0

mc.join()

### Test with threading (instance methods) -> Works!

In [8]:
# experiment status: 1 - idle, 2 - running (recording, logging), 0 - stopped
status = mp.Value('i', 1)

mc = MicrophoneControllerInstance(status,cfg["microphones"])

mc.start()

In [9]:
status.value = 2

Audio input stream started.


In [10]:
import time

In [11]:
status.value = 0

In [12]:
mc.stop()

Microphones recording stopped
